In [1]:
import pandas as pd
import os
from tqdm import tqdm


In [2]:
# DATA_PATH = '../data/20240131-file-urls.csv'
DATA_PATH = '../data/20240131_file_urls_with_selected_study_ids.csv'
DATA_SAVE_PATH = '../data/genelab_data_in_csv_format.pickle'


## Reading data file with URLs associated with it

In [3]:
data = pd.read_csv(DATA_PATH)

## Selecting rows with URLs which has data in csv format

In [14]:
%%time

row_index_with_csv = []
for row_index in range(data.shape[0]):
    if '.csv' in data.iloc[row_index].url:
        row_index_with_csv.append(row_index)

data_in_csv_format = data.iloc[row_index_with_csv]
    
data_in_csv_format.drop_duplicates(subset=['url'], inplace=True)

data_in_csv_format = data_in_csv_format[data_in_csv_format.url.str.contains('differential_expression.csv')]
data_in_csv_format = data_in_csv_format.dropna(subset=['datatype'])


CPU times: user 927 ms, sys: 5.38 ms, total: 932 ms
Wall time: 930 ms


<timed exec>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [15]:

# data_downloaded = pd.read_csv(data_in_csv_format.iloc[row_index].url)


data_in_csv_format.category.unique()

array(['GeneLab Processed RNA-Seq Files',
       'GeneLab Processed Microarray Data Files'], dtype=object)

## Downloading data

In [16]:
%%time

data_list = []
for row_index in tqdm(range(data_in_csv_format.shape[0])):
    data_dict = data_in_csv_format.iloc[row_index].to_dict()
    data_downloaded = pd.read_csv(data_in_csv_format.iloc[row_index].url)
    data_package = {
        'metadata' : data_dict,
        'data' : data_downloaded
    }
    data_list.append(data_package)


100%|███████████████████████████████████████████| 74/74 [18:16<00:00, 14.82s/it]

CPU times: user 2min 15s, sys: 2min 14s, total: 4min 29s
Wall time: 18min 16s


In [17]:
%%time

import pickle



with open(DATA_SAVE_PATH, 'wb') as f:
    pickle.dump(data_list, f)
    

CPU times: user 2.71 s, sys: 5.41 s, total: 8.12 s
Wall time: 13.9 s
